In [2]:
pip install python-docx


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 3.4 MB/s eta 0:00:00


In [4]:
from docx import Document
import pandas as pd
import re

# Replace with your uploaded file name
file_path = "/content/Prediction.docx"
label = "Prediction"  # ← set your label based on file name or content

doc = Document(file_path)

data = []
sl_no = 1

for para in doc.paragraphs:
    query = para.text.strip()
    if query:
        query_clean = re.sub(r"^\d+\.\s*", "", query)  # remove leading numbers like "1. "
        data.append({"sl.no": sl_no, "query": query_clean, "label": label})
        sl_no += 1

df = pd.DataFrame(data)
df.to_csv("labeled_queries.csv", index=False)

print("✅ Done! Saved as labeled_queries.csv")


✅ Done! Saved as labeled_queries.csv


In [6]:
from docx import Document
import pandas as pd
import re

# Replace with your uploaded file name
file_path = "/content/Compare.docx"
label = "Comparison"  # ← set your label based on file name or content

doc = Document(file_path)

data = []
sl_no = 1

for para in doc.paragraphs:
    query = para.text.strip()
    if query:
        query_clean = re.sub(r"^\d+\.\s*", "", query)  # remove leading numbers like "1. "
        data.append({"sl.no": sl_no, "query": query_clean, "label": label})
        sl_no += 1

df = pd.DataFrame(data)
df.to_csv("Compare_label.csv", index=False)

print("✅ Done! Saved as labeled_queries.csv")

✅ Done! Saved as labeled_queries.csv


In [7]:
from docx import Document
import pandas as pd
import re

# Replace with your uploaded file name
file_path = "/content/Explanatory.docx"
label = "Explanatory"  # ← set your label based on file name or content

doc = Document(file_path)

data = []
sl_no = 1

for para in doc.paragraphs:
    query = para.text.strip()
    if query:
        query_clean = re.sub(r"^\d+\.\s*", "", query)  # remove leading numbers like "1. "
        data.append({"sl.no": sl_no, "query": query_clean, "label": label})
        sl_no += 1

df = pd.DataFrame(data)
df.to_csv("Explanatory_label.csv", index=False)

print("✅ Done! Saved as labeled_queries.csv")

✅ Done! Saved as labeled_queries.csv


In [8]:
from docx import Document
import pandas as pd
import re

# Replace with your uploaded file name
file_path = "/content/Navigation.docx"
label = "Navigation"  # ← set your label based on file name or content

doc = Document(file_path)

data = []
sl_no = 1

for para in doc.paragraphs:
    query = para.text.strip()
    if query:
        query_clean = re.sub(r"^\d+\.\s*", "", query)  # remove leading numbers like "1. "
        data.append({"sl.no": sl_no, "query": query_clean, "label": label})
        sl_no += 1

df = pd.DataFrame(data)
df.to_csv("Navigation_label.csv", index=False)

print("✅ Done! Saved as labeled_queries.csv")

✅ Done! Saved as labeled_queries.csv


In [9]:
from docx import Document
import pandas as pd
import re

# Replace with your uploaded file name
file_path = "/content/Valuation.docx"
label = "Valuation"  # ← set your label based on file name or content

doc = Document(file_path)

data = []
sl_no = 1

for para in doc.paragraphs:
    query = para.text.strip()
    if query:
        query_clean = re.sub(r"^\d+\.\s*", "", query)  # remove leading numbers like "1. "
        data.append({"sl.no": sl_no, "query": query_clean, "label": label})
        sl_no += 1

df = pd.DataFrame(data)
df.to_csv("Valuation_label.csv", index=False)

print("✅ Done! Saved as labeled_queries.csv")

✅ Done! Saved as labeled_queries.csv


In [10]:
import pandas as pd
import glob

# Step 1: List all uploaded CSV files in Colab's working directory
csv_files = glob.glob("/content/*.csv")

# Step 2: Read and concatenate all CSVs
dfs = [pd.read_csv(file) for file in csv_files]
merged_df = pd.concat(dfs, ignore_index=True)

# Step 3: Drop extra columns like sl.no if they exist
if 'sl.no' in merged_df.columns:
    merged_df = merged_df.drop(columns=['sl.no'])

# Step 4: Save final merged CSV
merged_df.to_csv("all_queries_labeled.csv", index=False)
print("✅ Merged file saved as all_queries_labeled.csv")


✅ Merged file saved as all_queries_labeled.csv


In [11]:
merged_df.head()


,query,label
0,How does the price of LAND compare to the pric...,Valuation
1,What factors should I consider when determinin...,Valuation
2,How can I estimate the future value of a LAND ...,Valuation
3,How do road proximity and nearby landmarks aff...,Valuation
4,What is the impact of being near popular areas...,Valuation


In [2]:
!pip install -q transformers scikit-learn

import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from torch.optim import AdamW  # ✅ this is the correct source now
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tqdm import tqdm


In [3]:
df = pd.read_csv("/content/all_queries_labeled.csv")

le = LabelEncoder()
df["label_encoded"] = le.fit_transform(df["label"])
num_labels = len(le.classes_)

train_texts, temp_texts, train_labels, temp_labels = train_test_split(
    df["query"], df["label_encoded"], test_size=0.3, stratify=df["label_encoded"], random_state=42)

val_texts, test_texts, val_labels, test_labels = train_test_split(
    temp_texts, temp_labels, test_size=1/3, stratify=temp_labels, random_state=42)


In [9]:
class QueryDataset(Dataset):
    def __init__(self, texts, labels):
        self.labels = labels
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=128)
        assert len(self.encodings["input_ids"]) == len(self.labels), \
            f"Encoding and label size mismatch: {len(self.encodings['input_ids'])} vs {len(self.labels)}"

    def __getitem__(self, idx):
        try:
            item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
            item["labels"] = torch.tensor(self.labels[idx])
            return item
        except IndexError:
            print(f"❌ IndexError at idx={idx}")
            raise

    def __len__(self):
        return len(self.labels)


In [12]:
# Fix: wrap with list() to ensure correct format
train_texts = list(train_texts)
val_texts = list(val_texts)
test_texts = list(test_texts)

train_labels = list(train_labels)
val_labels = list(val_labels)
test_labels = list(test_labels)



In [13]:
train_dataset = QueryDataset(train_texts, train_labels)
val_dataset = QueryDataset(val_texts, val_labels)
test_dataset = QueryDataset(test_texts, test_labels)


In [14]:
from transformers import DistilBertForSequenceClassification
from torch.optim import AdamW

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=len(le.classes_)  # ← number of unique labels
).to(device)

optimizer = AdamW(model.parameters(), lr=5e-5)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
from torch.utils.data import DataLoader
from tqdm import tqdm

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        total_loss += loss.item()
    print(f"✅ Epoch {epoch+1} completed. Average loss: {total_loss / len(train_loader):.4f}")


Epoch 1/5: 100%|██████████| 449/449 [00:26<00:00, 16.78it/s]


✅ Epoch 1 completed. Average loss: 1.0706


Epoch 2/5: 100%|██████████| 449/449 [00:24<00:00, 18.24it/s]


✅ Epoch 2 completed. Average loss: 0.7461


Epoch 3/5: 100%|██████████| 449/449 [00:24<00:00, 18.38it/s]


✅ Epoch 3 completed. Average loss: 0.5444


Epoch 4/5: 100%|██████████| 449/449 [00:24<00:00, 18.53it/s]


✅ Epoch 4 completed. Average loss: 0.3667


Epoch 5/5: 100%|██████████| 449/449 [00:24<00:00, 18.02it/s]

✅ Epoch 5 completed. Average loss: 0.2528


In [16]:
test_loader = DataLoader(test_dataset, batch_size=8)
model.eval()

all_preds, all_labels = [], []

with torch.no_grad():
    for batch in test_loader:
        inputs = {k: v.to(device) for k, v in batch.items() if k != "labels"}
        labels = batch["labels"].to(device)
        outputs = model(**inputs)
        preds = torch.argmax(outputs.logits, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())


In [17]:
from sklearn.metrics import classification_report

print("✅ Final Test Set Evaluation:\n")
print(classification_report(all_labels, all_preds, target_names=le.classes_))


✅ Final Test Set Evaluation:

              precision    recall  f1-score   support

  Comparison       0.76      0.70      0.73        53
 Explanatory       0.57      0.66      0.61       105
  Navigation       0.79      0.72      0.75       103
  Prediction       0.89      0.76      0.82       101
   Valuation       0.65      0.69      0.67       151

    accuracy                           0.70       513
   macro avg       0.73      0.70      0.71       513
weighted avg       0.72      0.70      0.71       513



In [18]:
# Save model and tokenizer
model.save_pretrained("/content/drive/MyDrive/distilbert_faq_model")
tokenizer.save_pretrained("/content/drive/MyDrive/distilbert_faq_model")


('/content/drive/MyDrive/distilbert_faq_model/tokenizer_config.json',
 '/content/drive/MyDrive/distilbert_faq_model/special_tokens_map.json',
 '/content/drive/MyDrive/distilbert_faq_model/vocab.txt',
 '/content/drive/MyDrive/distilbert_faq_model/added_tokens.json',
 '/content/drive/MyDrive/distilbert_faq_model/tokenizer.json')